In [17]:
from sklearn.datasets import fetch_covtype
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (fetch_covtype())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])

y = to_categorical(y)
print(y[:10])

[[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]


In [16]:
print(x.shape,y.shape) 

print(np.unique(y)) # [1 2 3 4 5 6 7]
# 불균형확인
# 참고로 이 y 값은 0 클래스가 없기 대문에 0 을 지워줘야합니다 ... ㅂㄷㅂㄷ

print(np.unique(y,return_counts=True)) # [211840, 283301,  35754,   2747,   9493,  17367,  20510




(581012, 54) (581012, 8)
[0. 1.]
(array([0., 1.], dtype=float32), array([4067084,  581012], dtype=int64))
[[[1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]]]


In [8]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(
                                                    x,
                                                    y,
                                                    train_size=0.8,
                                                    shuffle = True,
                                                    #  stratify 는 데이터 불균형을 해결해줌
                                                    stratify=y,
                                                    random_state=21
                                                   )
# print(x.columns)
# y 를 원 핫 인코딩 변환

print(y_train[:10])
print("y_train shape : ",y_train.shape)
print("y_test shape : ",y_test.shape)

        0
258339  2
24249   1
543211  1
106441  2
454449  1
73360   2
52121   2
335819  2
469444  1
531881  1
y_train shape :  (464809, 1)
y_test shape :  (116203, 1)


In [9]:
model = Sequential([
    Dense(128,activation="relu",input_shape=(54,)),
    Dense(128,activation="relu") ,
    Dense(128,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(32,activation="relu") ,
    Dense(32,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(7,activation="softmax")
])

In [10]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
patience=10, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="sparse_categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=1, 
                 validation_split=0.2,batch_size=256,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


1453/1453 [==============================] - 6s 4ms/step - loss: nan - accuracy: 4.0339e-05 - val_loss: nan - val_accuracy: 0.0000e+00


In [11]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


3632/3632 [==============================] - 7s 2ms/step - loss: nan - accuracy: 0.0000e+00
loss: nan 
acc : 0.0


In [12]:
from sklearn.metrics import accuracy_score

y_predict = model.predict(x_test)
print(y_predict)
y_predict = np.argmax(y_predict,axis=1)


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [13]:
print(y_predict)

y_predict = tf.one_hot(y_predict,depth =7)


[0 0 0 ... 0 0 0]


In [14]:
# 데이터 출력

print("예측데이터 : ",y_predict[:20])
print("실제데이터 : ",y_test[:20])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)


예측데이터 :  tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]], shape=(20, 7), dtype=float32)
실제데이터 :          0
507183  1
108733  2
412166  1
287254  1
80040   2
193520  1
152947  1
406059  1
192875  2
528955  1
379642  2
559620  2
288780  3
331694  2
166276  2
370790  2
100861  2
356005  2
374055  2
81191   1


ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [ ]:
model.save("./_data\\"+"fetch(val_acc=0.89).h5")